### Work In Progress

In [2]:
import tweepy
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re
import string
#nltk.download('stopwords')

consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

Original tweets

In [22]:
searched_tweets = [tweet for tweet in tweepy.Cursor(api.search, q='craftbeer', lang='en').items(10)]
orig_tweets = [w.text for w in searched_tweets]
orig_tweets

['Brewing Solutions, by TERMOINOX® https://t.co/ZjZnJKy1Pj _ #brewing #cervejaartesanal #craftbeer #brewery… https://t.co/g2RgzbyiOY',
 'RT @BeertodayBT: 🎙NEW #btbt #podcast #episode Alert!\nWe hang with one of Queens Finest breweries @FifthHammerBeer to talk about the genesis…',
 'Cloudwater Tap Takeover Time! 🎉👌👀🍺 #cloudwater #taptakeover #beer #craftbeer #IPA #DIPA #TIPA #IPL #DDH #BBC… https://t.co/7IKJ3g4Lcr',
 'Danny Burke live tonight from 8pm! #livemusic #local #singersongwriter #Kilkenny #gig #Craftbeer https://t.co/mDyhXKcFYD',
 'RT @BeertodayBT: 🎙NEW #btbt #podcast #episode Alert!\nWe hang with one of Queens Finest breweries @FifthHammerBeer to talk about the genesis…',
 "RT @zappafaye: I went shopping tonight and couldn't be happier. @ArtifactBeer is amazeballs 😍😍 #craftbeer #ohiobeer  🍻\n@badhopper @JacobGri…",
 'RT @IndyCraftBrew: Fresh Local Indiana Craft Beer \n👉@18thStreetBrew \n\nBait and Click review over at \n\n👉 @moskihomebrew \n\n🍺#indianacraftbeer…',
 '🎙NEW #btbt

Function to get cleaned up text

In [21]:
def get_clean_tweets(query, max_tweets):
    searched_tweets = [tweet for tweet in tweepy.Cursor(api.search, q=query, lang='en').items(max_tweets)]
    clean_tweets = []
    
    for tweet in searched_tweets:
        tweet_text = tweet.text
        lower = tweet_text.lower()
        link = re.sub(r'https\S+', ' ', lower)
        punc = re.sub(r"[,.:;'\"’“”#@%-?!&$]+\ *", ' ', link)
        misc = re.sub('rt|…|amp', ' ', punc)
        pattern = re.compile(r"(.)\1{2,}")
        reduce_leng = pattern.sub(r"\1\1", misc)
        emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F" 
                           u"\U0001F300-\U0001F5FF"  
                           u"\U0001F680-\U0001F6FF"  
                           u"\U0001F1E0-\U0001F1FF"  
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           u"\U0001F900-\U0001F999"    
                           "]+", flags=re.UNICODE)
        emoji = emoji_pattern.sub(r'', reduce_leng)
        tokenizer = TweetTokenizer()
        tokens = tokenizer.tokenize(emoji) 
        stop_words = [w for w in tokens if not w in stopwords.words('english')]
        clean_tweets.append(stop_words)
    return clean_tweets

In [23]:
data = get_clean_tweets('craftbeer', 100)
print(data[:25])

[['brewing', 'solutions', 'termoinox', '®', '_', 'brewing', 'cervejaa', 'esanal', 'craftbeer', 'brewery'], ['bee', 'odaybt', 'new', 'btbt', 'podcast', 'episode', 'ale', 'hang', 'one', 'queens', 'finest', 'breweries', 'fifthhammerbeer', 'talk', 'genesis'], ['cloudwater', 'tap', 'takeover', 'time', 'cloudwater', 'taptakeover', 'beer', 'craftbeer', 'ipa', 'dipa', 'tipa', 'ipl', 'ddh', 'bbc'], ['danny', 'burke', 'live', 'tonight', 'pm', 'livemusic', 'local', 'singersongwriter', 'kilkenny', 'gig', 'craftbeer'], ['bee', 'odaybt', 'new', 'btbt', 'podcast', 'episode', 'ale', 'hang', 'one', 'queens', 'finest', 'breweries', 'fifthhammerbeer', 'talk', 'genesis'], ['zappafaye', 'went', 'shopping', 'tonight', 'happier', 'ifactbeer', 'amazeballs', 'craftbeer', 'ohiobeer', 'badhopper', 'jacobgri'], ['indycraftbrew', 'fresh', 'local', 'indiana', 'craft', 'beer', 'thstreetbrew', 'bait', 'click', 'review', 'moskihomebrew', 'indianacraftbeer'], ['new', 'btbt', 'podcast', 'episode', 'ale', 'hang', 'one', 

Frequency of words

In [24]:
import collections

data = [item for sublist in data for item in sublist]
counter=collections.Counter(data)
sorted_words = sorted(counter.items(), key=lambda x:x[1], reverse=True)
sorted_words[:20]

[('craftbeer', 47),
 ('beer', 28),
 ('craft', 14),
 ('new', 13),
 ('pm', 11),
 ('tonight', 7),
 ('review', 7),
 ('week', 7),
 ('bar', 7),
 ('caringtonarms', 7),
 ('open', 6),
 ('friday', 6),
 ('get', 6),
 ('another', 6),
 ('brewery', 5),
 ('episode', 5),
 ('bbc', 5),
 ('local', 5),
 ('like', 5),
 ('thursday', 5)]

Most common bigrams

In [27]:
b_gram = nltk.bigrams(data)
fdist = nltk.FreqDist(bgs)

b_gram_sorted = sorted(fdist.items(), key=lambda x:x[1], reverse=True)
b_gram_sorted[:15]

[(('craft', 'beer'), 8),
 (('caringtonarms', 'usual'), 5),
 (('usual', 'fish'), 5),
 (('fish', 'chip'), 5),
 (('chip', 'deal'), 5),
 (('deal', 'friday'), 5),
 (('friday', 'evening'), 5),
 (('evening', 'craftbeer'), 5),
 (('craftbeer', 'batter'), 5),
 (('batter', 'handmade'), 5),
 (('handmade', 'chips'), 5),
 (('chips', 'nothing'), 5),
 (('nothing', 'fear'), 5),
 (('citra', 'bbc'), 4),
 (('bbc', 'abv'), 4)]